In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
from sklearn.model_selection import GridSearchCV, train_test_split, KFold

import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from scipy import stats
import glob
import time
import datetime
import os
from scipy.stats import ttest_ind_from_stats
import matplotlib.pyplot as plt
import pickle
import time
import copy

%matplotlib inline

/Users/jonathanjanke/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/jonathanjanke/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
path = "351_Data/ILSVRC2012_Validation/Intermediate/"

In [3]:
predefined_folder_name = ""

If no predefined folder name is set, the latest data is used (folder with latest timestamp)

In [4]:
if predefined_folder_name == "":
    folder_list = np.sort(glob.glob(path + "/*"))
    curr_path = folder_list[-1] + "/"
else:
    curr_path = path + predefined_folder_name + "/"

In [5]:
curr_path

'351_Data/ILSVRC2012_Validation/Intermediate/20181010_0850/'

Determine the dataset

In [6]:
dataset_name = curr_path.split("/")[1].split("_")[0]

In [54]:
model_names = [fn.split("/")[-1] for fn in glob.glob(curr_path + "*")]

In [55]:
dataset_name

'ILSVRC2012'

In [56]:
model_names

['Inception ResNet', 'InceptionV3', 'Xception']

In [10]:
model_names = model_names[1:]

#### The following line is only inteded for the full model, assuming a class split of 10 classes per batch has been made

In [58]:
# for full model
use = False
if use:
    for fn in model_names:
        train_data_parts = glob.glob(curr_path + str(fn) + "\\train*")
        test_data_parts = glob.glob(curr_path + str(fn) + "\\test*")
        curr_train = []
        curr_test = []
        for train_part, test_part in zip(train_data_parts, test_data_parts):
            curr_train.append (np.load(train_part))
            #print (np.load(train_part)[10][1])
            #curr_test.append(np.load(test_part))

        train_data.append(np.concatenate(curr_train))
        #test_data.append(np.concatenate(curr_test))

#### The following line is for running the model from only one data file

In [59]:
train_data = []
test_data = []
for fn in model_names:
    train_data_parts = np.load(glob.glob(curr_path + str(fn) + "/train*")[0])
    for tt in glob.glob(curr_path + str(fn) + "/train*")[1:]:
        train_data_parts = np.vstack((train_data_parts,np.load(tt)))
    train_data.append(copy.copy(train_data_parts))
    
    test_data_parts = np.load(glob.glob(curr_path + str(fn) + "/test*")[0])
    for tt in glob.glob(curr_path + str(fn) + "/test*")[1:]:
        test_data_parts = np.vstack((test_data_parts, np.load(tt)))
    test_data.append(copy.copy(test_data_parts))

dataset dimensions

In [60]:
for ind,name in enumerate(model_names):
    print("" + name + " dimensions: " + str(train_data[ind][0][0].shape))

Inception ResNet dimensions: (98304,)
InceptionV3 dimensions: (131072,)
Xception dimensions: (204800,)


In [61]:
def iterate_over_datasets(datasets):
    ret_data = []
    for data in datasets:
        # rem here
        n_data = split_input_target(data)
        ret_data.append(n_data)
    return ret_data

In [62]:
def split_input_target (data):
    target = []
    inp = []
    for d in data:
        target.append(d[1][0])
        inp.append(d[0])
    return [np.array(inp), target]

In [63]:
# temp: only test purposes; remember to remove from iterate_over_datasets
import random
def split_input_target_special (data):
    target = []
    inp = []
    for d in data:
        target.append(random.randrange(0,2))
        inp.append(d[0])
    return [np.array(inp), target]

In [64]:
train_data = iterate_over_datasets(train_data)

In [65]:
test_data = iterate_over_datasets(test_data)

In [66]:
def reindex_classes(data):
    for ind, ds in enumerate(data):
        ds[1] = [x - min (ds[1]) for x in ds[1]]
        data[ind] = ds
    return data

In [67]:
train_data = reindex_classes(train_data)
test_data = reindex_classes(test_data)

In [68]:
def evaluate(model, data):
    y_pred = model.predict(data[0])
    
    try: y_proba = model.predict_proba(data[0])
    except: y_proba = model.predict(data[0])
    
    try: predictions = [round(value) for value in y_pred]
    except: predictions = [np.argmax(value) for value in y_pred]
    
    # evaluate predictions
    d = {}
    d["accuracy"] = accuracy_score(data[1], predictions)
    d["top-2-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=2)
    d["top-5-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=5)
    d["top-10-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=10)
    d["top-20-accuracy"] = get_top_n_accuracy(data[1], y_proba, n=20)
    d["confusion matrix"] = confusion_matrix(data[1], predictions)
    d["precision"] = precision_score(data[1], predictions, average='macro')
    d["recall"] = recall_score(data[1], predictions, average='macro')
    d["f1-score"] = f1_score(data[1], predictions, average='macro')
    # d["roc-auc"] = roc_auc_score(data[1], predictions, )
    d["cohen's kappa"] = cohen_kappa_score(data[1], predictions)
    
    return d

In [69]:
def get_measures(measure, data_type="data", measure_name="Accuracy"):
    return "%s in %s: %.2f" % (measure_name, data_type, measure)

In [70]:
def print_measures(evaluation, t):
    for key in evaluation.keys():
        if key!="confusion matrix":
            print (get_measures(evaluation[key], t, key))
        else:
            print(key)
            print(evaluation[key])

In [71]:
def write_to_file(train_eval, test_eval, model_stats, model_key, model_name, output_file_name):
    file_stamp = curr_path.split("/")[-2]
    train_size = str(len(train_data[0][0]))
    test_size = str(len(test_data[0][0]))
    dataset_dim = str(np.max(train_data[0][1] + test_data[0][1]) + 1)
    with open(output_file_name + '_train.csv', 'a') as f:
        print (output_file_name + '_train.csv')
        f.write("\n" + model_name + ',' + file_stamp + ',' + train_size + ',' + dataset_dim + ',' + model_key + ",")
        f.write(','.join([str(train_eval[x]) for x in train_eval.keys() if x!="confusion matrix"]) + ",")
        f.write(','.join([str(x) for x in model_stats["train"].values()]))
    
    with open(output_file_name + '_test.csv', 'a') as f:
        f.write("\n" + model_name + ',' + file_stamp + ',' + test_size + ',' + dataset_dim + ',' + model_key + ",")
        f.write(','.join([str(test_eval[x]) for x in test_eval.keys() if x!="confusion matrix"]) + ",")    
        f.write(','.join([str(x) for x in model_stats["test"].values()]))

In [72]:
def create_data_files(ind,prefix):
    path = "352_Benchmark_Results/" + dataset_name + "/"
    output_file_name = path + prefix + "_" + str(ind)
    if not os.path.exists(path):
        os.makedirs(path)
    with open(output_file_name + '_train.csv', 'w') as f:
        f.write('Dataset,Dataset Stamp,Dataset Size,# Dataset Classes,Model Architecture' + ',' + "accuracy" + ',' + "top-2-accuracy" + ',' + "top-5-accuracy" + ',' + "top-10-accuracy" + ',' + "top-20-accuracy" + ',' + "precision" + ',' + "recall" + ',' + "f1-score" + ',' + "cohen's kappa" + ',' + "Mean Accuracy over cross validation" + ',' + "mean standard deviation over cross validation" + ',' + "number observations over cross validation")
    with open(output_file_name + '_test.csv', 'w') as f:
        f.write('Dataset,Dataset Stamp,Dataset Size,# Dataset Classes,Model Architecture' + ',' + "accuracy" + ',' + "top-2-accuracy" + ',' + "top-5-accuracy" + ',' + "top-10-accuracy" + ',' + "top-20-accuracy" + ',' + "precision" + ',' + "recall" + ',' + "f1-score" + ',' + "cohen's kappa" + ',' + "Mean Accuracy over cross validation" + ',' + "mean standard deviation over cross validation" + ',' + "number observations over cross validation")
    return output_file_name

In [73]:
def get_data_from_sources (training_src, test_src, validation_src=""):
    train = get_data_from_source(training_src)
    test = get_data_from_source(test_src)
    try:
        if validation_src !="":
            val = get_data_from_source(validation_src)
        else:
            inp_test, inp_val, target_test, target_val = train_test_split(*test)
            test = (inp_test, target_test)
            val = (inp_val, target_val)
    except: 
        inp_test, inp_val, target_test, target_val = train_test_split(*test)
        test = (inp_test, target_test)
        val = (inp_val, target_val)
    return train, val, test

In [74]:
def create_model_A (input_shape, num_classes, learning_rate = 0.0001):
    model = Sequential()
    model.add(Dense(1024, input_shape=input_shape))
    model.add(Activation('elu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model

In [75]:
def create_model_B (input_shape, num_classes, learning_rate = 0.0001):
    model = Sequential()
    model.add(Dense(512, input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model

In [76]:
def create_model_C (input_shape, num_classes, learning_rate = 0.0001):
    model = Sequential()
    model.add(Dense(num_classes, input_shape=input_shape))
    model.add(Activation('softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=learning_rate),
                  metrics=['accuracy'])
    return model

In [77]:
def create_models(input_shape, num_classes, learning_rate = 0.0001):
    mlp_models = []
    mlp_models.append(create_model_A(input_shape, num_classes, learning_rate))
    mlp_models.append(create_model_B(input_shape, num_classes, learning_rate))
    mlp_models.append(create_model_C(input_shape, num_classes, learning_rate))
    return mlp_models

In [78]:
# MLP hyperparameters
epochs=[10]
learning_rate= [0.001, 0.0001]

In [79]:
import copy

mlp_models = []
for i in learning_rate:
    for j in range(3):
        mlp_models.append({})
num_classes = len(set(train_data[0][1]))
mlp_names = []

for tr_ind, tr_data in enumerate(train_data):
    t_ind = copy.deepcopy(tr_ind)
    input_shape = tr_data[0][0].shape
    for lr_ind, lr in enumerate(learning_rate):
        temp_models = create_models(input_shape, num_classes, lr)
        # for mi, mm in enumerate(mlp_models):
        for mi in range (len(temp_models)):
            ind = copy.deepcopy(mi)
            mlp_models[lr_ind * len(temp_models) + ind][model_names[t_ind]] = temp_models[ind]
            model_name = "MLP-{}_LR-{}".format(mi,lr)
            mlp_names.append(model_name)
            print (model_name)

MLP-0_LR-0.001
MLP-1_LR-0.001
MLP-2_LR-0.001
MLP-0_LR-0.0001
MLP-1_LR-0.0001
MLP-2_LR-0.0001
MLP-0_LR-0.001
MLP-1_LR-0.001
MLP-2_LR-0.001
MLP-0_LR-0.0001
MLP-1_LR-0.0001
MLP-2_LR-0.0001
MLP-0_LR-0.001
MLP-1_LR-0.001
MLP-2_LR-0.001
MLP-0_LR-0.0001
MLP-1_LR-0.0001
MLP-2_LR-0.0001


In [80]:
mlp_models

[{'Inception ResNet': <keras.engine.sequential.Sequential at 0x10775f160>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a97aed8d0>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a17498c50>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x107743fd0>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a98a70cf8>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a1754e278>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x1a1729f4e0>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a99348be0>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a176cdef0>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x1a66575b38>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a172d8cc0>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a17769be0>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x1a95235908>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a17

In [81]:
svm_candidates = [
  {'C': [1, 10], 'gamma': [0.001, 0.0001], 'kernel': ['rbf'], 'probability':[True]},
]

In [82]:
rfe_candidates = [
  {'n_estimators': [100, 1000], 'criterion': ['gini', "entropy"]}
]

In [83]:
adb_candidates = [
  {'n_estimators': [10, 100, 1000], 'learning_rate': [1.0, 0.5, 0.1]}
]

In [84]:
gbc_candidates = [
  {'n_estimators': [100], 'learning_rate': [0.1]}
]

In [85]:
lr_candidates= [
    {'penalty': ["l1", "l2"], 'C': [0.001,0.01,0.1,1,10,100]}
]

In [86]:
dtc_candidates= [
    {'max_depth':[5,10,50], 'min_samples_split':np.linspace(0.1, 0.8, 8, endpoint=True), 'min_samples_leaf':np.linspace(0.1, 0.5, 5, endpoint=True), 'max_features':[10,100,500]}
]

In [87]:
xgbc_candidates = [
    {
        'max_depth':[1,5,10], 'learning_rate':[0.01, 0.1], 'n_estimators':[10, 100]     # objective, booster
    }
]

In [88]:
knn_candidates = [
    {
        'n_neighbors':[1,10],
#        'leaf_size':[1,5],
    }  
]

In [89]:
parameter_candidates = {}
parameter_candidates ["svm"] = svm_candidates
parameter_candidates["rfe"] = rfe_candidates
parameter_candidates ["adb"] = adb_candidates
parameter_candidates["gbc"] = gbc_candidates
parameter_candidates["lr"] = lr_candidates
parameter_candidates ["dtc"] = dtc_candidates
parameter_candidates["xgbc"] = xgbc_candidates
parameter_candidates["knn"] = knn_candidates

In [90]:
len(mlp_models)

6

In [91]:
rfe = RandomForestClassifier()
adb = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
lr = LogisticRegression()
dtc = DecisionTreeClassifier()#
xgbc = XGBClassifier()
svc = svm.SVC()
knn = KNeighborsClassifier()

benchmark_models = {"rfe":rfe, "xgbc":xgbc, "adb":adb, "gbc":gbc, "lr":lr, "dtc":dtc, "svm":svc, "knn":knn}

for x in range(len(mlp_models)):
    benchmark_models[mlp_names[x]] = mlp_models[x]

In [92]:
benchmark_models

{'lr': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 'svm': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'knn': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=5, p=2,
            weights='uniform')}

In [93]:
def get_model_stats(cv_results, best_params, folds):     
    truth = [param == best_params for param in cv_results['params']]
            
    curr_model_stats = {}
    curr_model_stats["train"] = {}
    curr_model_stats["train"]["mean"] = cv_results['mean_train_score'][truth][0]
    curr_model_stats["train"]["std_dev"] = cv_results['std_train_score'][truth][0]
    curr_model_stats["train"]["observations"] = folds
    curr_model_stats["test"] = {}
    curr_model_stats["test"]["mean"] = cv_results['mean_test_score'][truth][0]
    curr_model_stats["test"]["std_dev"] = cv_results['std_test_score'][truth][0]
    curr_model_stats["test"]["observations"] = folds
    print("Mean: {}".format(cv_results['mean_test_score'][truth]))
    print("Standard deviation: {}".format(cv_results['std_test_score'][truth]))
    print("Number of observations: {}".format(folds))
    return curr_model_stats

In [94]:
import statistics

In [95]:
def plot_model_stats(hist):
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [96]:
def grid_search_model(benchmark_models, parameter_candidates, train, test,model_name, output_file_name, folds = 4, prefix = "00000000"):
    model_stats = {}
    #try:
    for model_key in benchmark_models.keys():
        model = benchmark_models[model_key]
        print ((" Model: " + str(model_key)+ " ").center(30, '#'))
        start_time = time.time()
        # try:
        try:
            clf = GridSearchCV(estimator=model, param_grid=parameter_candidates[model_key], n_jobs=4, cv=folds,return_train_score=True)
            clf.fit(*train)
            model = clf.best_estimator_
            print ("Best parameters: {}".format(clf.best_params_))
            path = "352_Benchmark_Results/" + prefix + "_" + dataset_name + "/" + model_name + "/"
            print(path)
            if not os.path.exists(path):
                os.makedirs(path)
            filename = path + model_key + ".sav"

            try:
                pickle.dump(clf, open(filename, 'wb'))
            except:
                print(filename + " could not be created.")
            curr_model_stats = get_model_stats(clf.cv_results_, clf.best_params_, folds)
        except:
            model = benchmark_models[model_key][model_name]
            kf = KFold(n_splits=folds)
            cv_results = {}
            target_cat = np.array(to_categorical(train[1]))
            # print (target_cat.shape)
            cv_results["train"] = []
            cv_results["test"] = []
            curr_model_stats = {
                    "train": {
                        "mean":0,
                        "std_dev":0,
                        "observations":0
                    },
                    "test": {
                        "mean":0,
                        "std_dev":0,
                        "observations":0
                    }
                }
            Wsave = model.get_weights()
            for epoch in epochs:
                for kfold_ind, indices in enumerate(kf.split(train[0])):
                    model.set_weights(Wsave)
                    train_index, test_index = indices
                    X_train, X_test = train[0][train_index], train[0][test_index]
                    y_train, y_test = target_cat[train_index], target_cat[test_index]
                    # print(y_train)
                    # tmp_model = keras.models.clone_model(model)
                    # tmp_model.compile(loss=keras.losses.categorical_crossentropy,
                    #   optimizer=keras.optimizers.Adam(lr=learning_rate),
                    #   metrics=['accuracy'])
                    history = model.fit(X_train, y_train, epochs = epoch, validation_data = (X_test, y_test))
                    path = "352_Benchmark_Results/" + prefix + "_" + dataset_name + "/" + model_name + "/"
                    if not os.path.exists(path):
                        os.makedirs(path)
                    model.save_weights(path + str(kfold_ind) + "_" + model_key + ".h5")

                    print(model_key)
                    plot_model_stats(history)
                    train_res = model.evaluate(X_train, y=y_train)[1]
                    test_res = model.evaluate(X_test, y=y_test)[1]
                    cv_results["train"].append(train_res)
                    cv_results["test"].append(test_res)
                # benchmark_models[model_key] = model
                if curr_model_stats["test"]["mean"] < statistics.mean(cv_results["test"]):
                    curr_model_stats = {
                        "train": {
                            "mean":statistics.mean(cv_results["train"]),
                            "std_dev":statistics.stdev(cv_results["train"]),
                            "observations":len(cv_results["train"])
                        },
                        "test": {
                            "mean":statistics.mean(cv_results["test"]),
                            "std_dev":statistics.stdev(cv_results["test"]),
                            "observations":len(cv_results["test"])
                        }
                    }
        model_stats[model_key] = curr_model_stats           


        train_eval = evaluate(model, train)
        test_eval = evaluate(model, test)
        print_measures(train_eval, "Train")
        print_measures(test_eval, "Test")
        print("Total Time: {}".format(time.time() - start_time))
        write_to_file(train_eval, test_eval, curr_model_stats, model_key, model_name, output_file_name)

        # benchmark_models[model_key][model_name] = model
        # except:
        #    print(model_key)
    # except:
    #     print("Failed but not discontinued")
    #    pass
    return benchmark_models, model_stats

In [97]:
# input_data with shape (n,m)
def get_top_n_accuracy(input_target, input_data, n=5):
    count = 0
    inp_data = input_data.copy()
    for ind, pred in enumerate(inp_data):
        max_classes = []
        for i in range (n):
            max_classes.append(np.argmax(pred))
            pred[np.argmax(pred)]=-1
        if input_target[ind] in max_classes: 
            count += 1
    return count/len(input_target)

In [98]:
mlp_models

[{'Inception ResNet': <keras.engine.sequential.Sequential at 0x10775f160>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a97aed8d0>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a17498c50>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x107743fd0>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a98a70cf8>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a1754e278>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x1a1729f4e0>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a99348be0>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a176cdef0>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x1a66575b38>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a172d8cc0>,
  'Xception': <keras.engine.sequential.Sequential at 0x1a17769be0>},
 {'Inception ResNet': <keras.engine.sequential.Sequential at 0x1a95235908>,
  'InceptionV3': <keras.engine.sequential.Sequential at 0x1a17

In [101]:
for m,d in enumerate(mlp_models):
    for key, value in d.items():
        model_json = value.to_json()
        with open("352_Benchmark_Results/temp/{}_model_{}.json".format(key,m), "w") as json_file:
            json_file.write(model_json)

In [110]:
final_models = []
stats = {}

now = datetime.datetime.now()
prefix = now.strftime("%Y%m%d_%H%M")

for ind, _ in enumerate(train_data):
    
    print (("").center(60, '_'))
    print (("").center(60, '#'))
    print ((" Dataset Index: " + str(ind)+ " ").center(60, '#'))
    print (("").center(60, '#'))
    file_name = create_data_files(ind,prefix)
    m,stat = grid_search_model(benchmark_models, parameter_candidates, train_data[ind], test_data[ind], model_names[ind], file_name, 2, prefix)
    final_models.append(m)
    stats[model_names[ind]] = stat

____________________________________________________________
############################################################
##################### Dataset Index: 0 #####################
############################################################
######### Model: lr ##########
Best parameters: {'C': 0.001, 'penalty': 'l2'}
352_Benchmark_Results/20181029_1157_ILSVRC2012/InceptionV3/
Mean: [0.84586667]
Standard deviation: [0.00275314]
Number of observations: 2
accuracy in Train: 1.00
top-2-accuracy in Train: 1.00
top-5-accuracy in Train: 1.00
top-10-accuracy in Train: 1.00
top-20-accuracy in Train: 1.00
confusion matrix
[[34  0  0 ...  0  0  0]
 [ 0 34  0 ...  0  0  0]
 [ 0  0 41 ...  0  0  0]
 ...
 [ 0  0  0 ... 37  0  0]
 [ 0  0  0 ...  0 36  0]
 [ 0  0  0 ...  0  0 35]]
precision in Train: 1.00
recall in Train: 1.00
f1-score in Train: 1.00
cohen's kappa in Train: 1.00
accuracy in Test: 0.85
top-2-accuracy in Test: 0.93
top-5-accuracy in Test: 0.98
top-10-accuracy in Test: 0.99
top-20-acc

/Users/jonathanjanke/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jonathanjanke/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy in Train: 1.00
top-2-accuracy in Train: 0.79
top-5-accuracy in Train: 0.80
top-10-accuracy in Train: 0.80
top-20-accuracy in Train: 0.81
confusion matrix
[[34  0  0 ...  0  0  0]
 [ 0 34  0 ...  0  0  0]
 [ 0  0 41 ...  0  0  0]
 ...
 [ 0  0  0 ... 37  0  0]
 [ 0  0  0 ...  0 35  0]
 [ 0  0  0 ...  0  0 35]]
precision in Train: 1.00
recall in Train: 1.00
f1-score in Train: 1.00
cohen's kappa in Train: 1.00
accuracy in Test: 0.36
top-2-accuracy in Test: 0.62
top-5-accuracy in Test: 0.66
top-10-accuracy in Test: 0.70
top-20-accuracy in Test: 0.76
confusion matrix
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 5 ... 0 0 0]
 ...
 [0 0 0 ... 8 0 0]
 [0 0 0 ... 0 7 0]
 [0 0 0 ... 0 0 8]]
precision in Test: 0.79
recall in Test: 0.39
f1-score in Test: 0.46
cohen's kappa in Test: 0.35
Total Time: 50271.630492925644
352_Benchmark_Results/ILSVRC2012/20181029_1157_0_train.csv
######### Model: knn #########
Best parameters: {'n_neighbors': 1}
352_Benchmark_Results/20181029_1157_ILSVRC2012/Inc

In [ ]:
benchmark_models

In [ ]:
# expects a dictionary object with the structure dataset -> classification_algorithm -> train/test -> mean/std_dev/observations
# returns a dictionary with each being (dataset:significance_matrix as pandas DF), e.g., sig_matrix_dict["InceptionV3"]
def get_signficance_matrix(stats, base="test", sig_level=0.05, one_sided = False):
    if one_sided:
        sig_level /= 2
    sig_matrix_dict = {}
    for dataset in stats.keys():
        sig_matrix_dict[dataset] = []
        model_entries = stats[dataset].keys()
        for model_key in stats[dataset].keys():
            dim_B = []
            for compare_model_key in stats[dataset].keys():
                t_stats = ttest_ind_from_stats(stats[dataset][model_key][base]["mean"], stats[dataset][model_key][base]["std_dev"], stats[dataset][model_key][base]["observations"], stats[dataset][compare_model_key][base]["mean"], stats[dataset][compare_model_key][base]["std_dev"], stats[dataset][compare_model_key][base]["observations"])
                if t_stats[1] < sig_level and t_stats[0] < 0.0:
                    dim_B.append(True)
                else:
                    dim_B.append(False)
            sig_matrix_dict[dataset].append(dim_B)
        sig_matrix_dict[dataset] = pd.DataFrame(sig_matrix_dict[dataset],index=model_entries,columns=model_entries)
    return sig_matrix_dict

In [ ]:
get_signficance_matrix(stats)